In [ ]:
import numpy as np
import scipy.io
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [ ]:
# Data Preprocessing
# Scale data to [0, 255] for consistency with RGB image channels
def scale_data(data):
    scaled_data = (data - np.min(data)) / (np.max(data) - np.min(data)) * 255
    return scaled_data.astype(np.uint8)

In [ ]:
# Data Preprocessing
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load datasets from Google Drive
narrow_dataset_path = '/content/drive/MyDrive/ASP_Project/Data/narrow_grid_dataset.mat'
wide_dataset_path = '/content/drive/MyDrive/ASP_Project/Data/wide_grid_dataset.mat'

data_narrow = scipy.io.loadmat(narrow_dataset_path)
data_wide = scipy.io.loadmat(wide_dataset_path)

# Extract data and labels
dataset_narrow = data_narrow['dataset_narrow']
labels_narrow = data_narrow['labels_narrow']

dataset_wide = data_wide['dataset_wide']
labels_wide = data_wide['labels_wide']

# Scale Data
dataset_narrow_scaled = scale_data(dataset_narrow)
dataset_wide_scaled = scale_data(dataset_wide)

# Split data into training and testing subsets (90% training, 10% testing)
x_train_narrow, x_test_narrow, y_train_narrow, y_test_narrow = train_test_split(dataset_narrow_scaled, labels_narrow, test_size=0.1, random_state=42)
x_train_wide, x_test_wide, y_train_wide, y_test_wide = train_test_split(dataset_wide_scaled, labels_wide, test_size=0.1, random_state=42)

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# Build Model
def build_model():
    inputs = tf.keras.Input(shape=(IMG_SIZE_Height, IMG_SIZE_Width, 3))

    # Convolutional Layers with Batch Normalization and ReLU activation
    x = layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=256, kernel_size=2, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=256, kernel_size=2, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=256, kernel_size=2, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Flatten the output from the convolutional layers
    x = layers.Flatten()(x)

    # Fully Connected Layers
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    # Output layer with Sigmoid activation for binary labels
    outputs = layers.Dense(labels_narrow.shape[1], activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    return model

In [ ]:
IMG_SIZE_Height, IMG_SIZE_Width = dataset_narrow.shape[1], dataset_narrow.shape[2]

model = build_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model
# Train the model for the narrow dataset
history_narrow = model.fit(x_train_narrow, y_train_narrow, epochs=200, batch_size=32, validation_data=(x_test_narrow, y_test_narrow))

# Train the model for the wide dataset
history_wide = model.fit(x_train_wide, y_train_wide, epochs=200, batch_size=32, validation_data=(x_test_wide, y_test_wide))

In [ ]:
# Noise addition
def add_noise(data, snr_db):
    signal_power = np.mean(data ** 2)
    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear
    noise = np.sqrt(noise_power) * np.random.randn(*data.shape)
    return data + noise

In [ ]:
# Model evaluation
snrs = [0, -5, -10, -15, -20]

# Evaluate the model for different SNR values and visualize the results
results = {}
for snr in snrs:
    noisy_data_narrow = add_noise(x_test_narrow, snr)
    predictions = model.predict(noisy_data_narrow)
    results[snr] = predictions

In [ ]:
# Generating results and plots
import matplotlib.pyplot as plt

for snr, predictions in results.items():
    fig, ax = plt.subplots(1, 2, figsize=(15, 6))
    ax[0].plot(y_test_narrow[:, 0], label='Real $\theta_1$', marker='^')
    ax[0].plot(predictions[:, 0], label='Predicted $\theta_1$', linestyle='--', marker='o')
    ax[1].plot(y_test_narrow[:, 1], label='Real $\theta_2$', marker='^')
    ax[1].plot(predictions[:, 1], label='Predicted $\theta_2$', linestyle='--', marker='o')
    ax[0].set_title(f'DOA Estimation Performance for SNR = {snr}')
    ax[1].set_title(f'DOA Estimation Error for SNR = {snr}')
    ax[0].legend()
    ax[1].legend()
    plt.show()

In [ ]:
# Calculating RMSE and plotting
from sklearn.metrics import mean_squared_error

rmse_values = {}
for snr, predictions in results.items():
    rmse_values[snr] = np.sqrt(mean_squared_error(y_test_narrow, predictions))

plt.figure(figsize=(10, 6))
plt.plot(snrs, [rmse_values[snr] for snr in snrs], marker='o')
plt.xlabel('SNR (dB)')
plt.ylabel('RMSE (Degrees)')
plt.title('RMSE for Different SNRs')
plt.show()